<a href="https://colab.research.google.com/github/Ash100/MDS/blob/main/SSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solvent-Accessible Surface Area (SASA) analysis

This notebook is designed by **Dr. Ashfaq Ahmad**, and can be used for research purposes. To watch a complete video demonstration, [click here](https://youtu.be/nKOb3MT5sMo)

We will perform these analysis using AmberTools.

Citation: https://pubs.acs.org/doi/10.1021/acs.jcim.3c01153

In [ ]:
#@title setting the environment
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title install AmberTools
!conda install -y -c conda-forge ambertools

In [2]:
#@title Install pytraj
!pip install -q pytraj

In [3]:
#@title connect you Gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pytraj as pt

# Example paths
dcd_path = "/content/drive/MyDrive/Dr_Zeeshan/test/prot_lig_prod1-20_whole.dcd"
pdb_path = "/content/drive/MyDrive/Dr_Zeeshan/test/prot_lig_equil.pdb"

# Load trajectory
traj = pt.load(dcd_path, top=pdb_path)

### Generating SASA.in file is not that simple to click. It needs a proper probing of ligand residue present in your complex. How to probe ligand effectively? For complete detail, [watch here](https://youtu.be/nKOb3MT5sMo)

In [ ]:
#@title writing sasa.in file
%%writefile cpptraj_sasa.in
parm /content/drive/MyDrive/Dr_Zeeshan/test/prot_lig_equil.pdb

# Load full system trajectory (protein + ligand + solvent)
trajin /content/drive/MyDrive/Dr_Zeeshan/test/prot_lig_prod1-20_whole.dcd
strip @H=
surf complex_sasa out complex_sasa.dat
run
clear all

# Protein only (residues 1 to 487)
parm /content/drive/MyDrive/Dr_Zeeshan/test/prot_lig_equil.pdb
trajin /content/drive/MyDrive/Dr_Zeeshan/test/prot_lig_prod1-20_whole.dcd
strip !:1-403
strip @H=
trajout protein.dcd
trajout protein.pdb pdb
run
clear all

parm protein.pdb
trajin protein.dcd
surf protein_sasa out protein_sasa.dat
run
clear all

# Ligand only (residue 404)
parm /content/drive/MyDrive/Dr_Zeeshan/test/prot_lig_equil.pdb
trajin /content/drive/MyDrive/Dr_Zeeshan/test/prot_lig_prod1-20_whole.dcd
strip !:404
strip @H=
trajout ligand.dcd
trajout ligand.pdb pdb
run
clear all

parm ligand.pdb
trajin ligand.dcd
surf ligand_sasa out ligand_sasa.dat
run

In [ ]:
#@title executing sasa.in in cpptraj
!cpptraj -i cpptraj_sasa.in


In [ ]:
#@title validating lengths
import numpy as np

sasa_protein = np.loadtxt("protein_sasa.dat", usecols=1)
sasa_ligand = np.loadtxt("ligand_sasa.dat", usecols=1)
sasa_complex = np.loadtxt("complex_sasa.dat", usecols=1)

frames_protein = np.arange(len(sasa_protein))
frames_ligand = np.arange(len(sasa_ligand))
frames_complex = np.arange(len(sasa_complex))

print("Lengths:")
print(f"Protein SASA: {len(sasa_protein)}")
print(f"Ligand SASA: {len(sasa_ligand)}")
print(f"Complex SASA: {len(sasa_complex)}")


In [ ]:
#@title plot protein SASA
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.plot(frames_protein, sasa_protein, color='steelblue', label='Protein SASA')
plt.xlabel("Frame")
plt.ylabel("SASA (Å²)")
plt.title("Protein SASA Over Time")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("/content/test_protein.png", dpi=600)
plt.show()


In [ ]:
#@title plot Ligand SASA
plt.figure(figsize=(8, 4))
plt.plot(frames_ligand, sasa_ligand, color='seagreen', label='Test Ligand SASA')
plt.xlabel("Frame")
plt.ylabel("SASA (Å²)")
plt.title("Ligand SASA Over Time")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("/content/ligand_sasa_plot.png", dpi=600)
plt.show()


In [11]:
min_len = min(len(sasa_protein), len(sasa_ligand), len(sasa_complex))
bsa = sasa_protein[:min_len] + sasa_ligand[:min_len] - sasa_complex[:min_len]


In [ ]:
#@title plot complex SASA
plt.figure(figsize=(8, 4))
plt.plot(frames_complex, sasa_complex, color='orange', label='Complex SASA')
plt.xlabel("Frame")
plt.ylabel("SASA (Å²)")
plt.title("Complex SASA Over Time")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("/content/complex_sasa_plot.png", dpi=600)
plt.show()


In [ ]:
#@title calculate BSA and plot
bsa = sasa_protein[:min_len] + sasa_ligand[:min_len] - sasa_complex[:min_len]

plt.figure(figsize=(8, 4))
plt.plot(np.arange(len(bsa)), bsa, color='crimson', label='Buried Surface Area (BSA)')
plt.xlabel("Frame")
plt.ylabel("SASA (Å²)")
plt.title("Buried Surface Area Over Time")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("/content/bsa_plot_600dpi.png", dpi=600)
plt.show()


I believe you learnt something new, and I did not waste time.

Please support my work by subscribing to [Bioinformatics Insights](https://youtube.com/@bioinformaticsinsights?si=ExMr3iD26LZYob7Q)